In [ ]:
!pip install openai
# Download dataset:
!gdown -O dataset.zip https://drive.google.com/uc?id=1JtRIbZDuZlBEA6vp870eW3ei0r32uYqF
# https://drive.google.com/file/d/1JtRIbZDuZlBEA6vp870eW3ei0r32uYqF/view?usp=sharing
!unzip -q dataset.zip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.7 MB/s eta 0:00:00
Downloading...
From (original): https://drive.google.com/uc?id=1JtRIbZDuZlBEA6vp870eW3ei0r32uYqF
From (redirected): https://drive.google.com/uc?id=1JtRIbZDuZlBEA6vp870eW3ei0r32uYqF&confirm=t&uuid=e22a9f93-3175-49f2-8254-755215504774
To: /content/dataset.zip
100% 852M/852M [00:05<00:00, 144MB/s]


In [ ]:
from openai import OpenAI
client = OpenAI(api_key = 'secret')

response = client.images.generate(
  model="dall-e-2",
  prompt="a man with black jacket",
  size="1024x1024",
  quality="standard",
  n=1,
)

image_url = response.data[0].url

# Show image with given url:
from IPython.display import Image
Image(url=image_url, width=256, height=256)

In [ ]:
import pandas as pd
# Read dataset/image_captions_cleaned captions with pandas:
dtype_dict = {'name': str, 'caption': str}
df2 = pd.read_csv('dataset/image_captions_cleaned.txt', sep=",", header=0, dtype=dtype_dict)
saved_captions = df2['caption'].tolist()

In [ ]:
from PIL import Image
import requests
from io import BytesIO
from tqdm.auto import tqdm
import os
from time import sleep

current_time = 'dalle2'

try:
  os.mkdir('inference/'+current_time)
except:
  try:
    os.mkdir('inference')
    os.mkdir('inference/'+current_time)
    print('Directory/s created')
  except:
    print('Directory/s already exists')

index = 128
number_gen = 256
for idx in tqdm(saved_captions[index:index+number_gen]):
    index += 1
    response = client.images.generate(
    model="dall-e-2",
    prompt=idx,
    size="1024x1024",
    quality="standard",
    n=1,
    )
    image_url = response.data[0].url
    response = requests.get(image_url)
    img = Image.open(BytesIO(response.content)).save(f'inference/{current_time}/{index:05d}.png')
    sleep(3)

print(str(len(os.listdir('/content/inference/'+current_time)))+' images are saved.')
print('Last image is: '+str(index+number_gen))

Directory/s already exists


  0%|          | 0/256 [00:00<?, ?it/s]

BadRequestError: Error code: 400 - {'error': {'code': 'content_policy_violation', 'message': 'Your request was rejected as a result of our safety system. Your prompt may contain text that is not allowed by our safety system.', 'param': None, 'type': 'invalid_request_error'}}

In [ ]:
!cd inference && zip -r dalle2.zip dalle2/
# To save inference output zip to google drive
from google.colab import drive
drive.mount('/content/drive')
!cp -r /content/inference/dalle2.zip /content/drive/MyDrive/dalle2.zip